## Document Loaders 알아보기

다양한 문서를 기반으로 LLM을 활용할 때, RAG에서는 문서를 불러오기 위해 하나의 객체를 새로 정의한다. 랭체인에서는 Document 객체라고 한다.  
Document 객체는 문서의 내용인 page_content와 문서의 위치, 제목, 페이지 번호 등 문서의 특성인 metadata를 포함한다.  
메타 데이터가 있어야 RAG시스템에 투입된 문서에 근거해서 답변해야 하니 검색이 빠르며, 필요한 정보만 찾고 싶은 필터링 기능에도 활용된다. 

In [6]:
# PyPDFLoader를 이용한 PDF 파일 로드 

# PyPDFLoader 불러오기
from langchain.document_loaders import PyPDFLoader 
filename = "data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf"

# PDF 파일 불러올 객체 PyPDFLoader 선언
loader = PyPDFLoader(filename)

# PDF 파일 로드 및 페이지별로 자르기 
pages = loader.load_and_split()
print(pages[5].page_content)

| 6 | CIS이슈리포트 2022-2호 
▶(주요품목① : 5G 이동통신) 정보통신 테마 내 기술분야 중 혁신성장 정책금융 공급규모가 가장 큰 차세대무선통신미디어 분야의 경우 4G/5G 기술품목의 정책금융 공급 비중이 가장 높은 것으로 확인됨[차세대무선통신미디어 분야 내 기술품목별 혁신성장 정책금융 공급액 추이](단위: 억 원)
▶5G 이동통신 시스템은 ITU(International Telecommunication Union)가 정의한 5세대 이동통신 규격을 만족시키는 무선 이동통신 네트워크 기술로, 2019년부터 국내 서비스를 시작함￮4G 이동통신 시스템(LTE)과 비교할 때 전송속도의 향상(1Gbps→20Gbps), 이동성 향상(350km/h→500km/h에서 끊김없는 데이터 전송 가능), 최대 연결가능 기기수 증가(10만 대 →100만 대 이상), 데이터 전송지연 감소(10ms→1ms) 등의 향상된 기능을 제공함￮5G는 전송속도 향상, 다수기기 접속 및 지연시간 단축을 위해 ①밀리미터파 통신이 가능한 주파수 확장, ②스몰셀(Small cell)을 도입한 기지국, ③다중안테나 송수신(Massive MIMO), ④네트워크 슬라이싱(Network Slicing) 등의 기술을 도입함[5G 주요 요소기술 특징]
자료: 삼정 KPMG


텍스트는 불러왔지만 차트+텍스트, 이미지+텍스트로 구성된 혼합 텍스트는 추출하지 못했다. 드래크가 가능한 부분만 추출했기 때문인데 OCR기능을 통해서 추출 가능하다.  
PyPDF의 rapidocr-onnxruntime 라이브러리를 사용하고 extract_images 매개변수를 true로 설정한다.

In [8]:
# PyPDFLoader 불러오기
from langchain.document_loaders import PyPDFLoader 

# PDF 파일 불러올 객체 PyPDFLoader 선언(extract_images=True으로 OCR수행)
loader = PyPDFLoader(filename, extract_images=True)

# PDF 파일 로드 및 페이지별로 자르기
pages = loader.load_and_split()
print(pages[5].page_content)

| 6 | CIS이슈리포트 2022-2호 
▶(주요품목① : 5G 이동통신) 정보통신 테마 내 기술분야 중 혁신성장 정책금융 공급규모가 가장 큰 차세대무선통신미디어 분야의 경우 4G/5G 기술품목의 정책금융 공급 비중이 가장 높은 것으로 확인됨[차세대무선통신미디어 분야 내 기술품목별 혁신성장 정책금융 공급액 추이](단위: 억 원)
▶5G 이동통신 시스템은 ITU(International Telecommunication Union)가 정의한 5세대 이동통신 규격을 만족시키는 무선 이동통신 네트워크 기술로, 2019년부터 국내 서비스를 시작함￮4G 이동통신 시스템(LTE)과 비교할 때 전송속도의 향상(1Gbps→20Gbps), 이동성 향상(350km/h→500km/h에서 끊김없는 데이터 전송 가능), 최대 연결가능 기기수 증가(10만 대 →100만 대 이상), 데이터 전송지연 감소(10ms→1ms) 등의 향상된 기능을 제공함￮5G는 전송속도 향상, 다수기기 접속 및 지연시간 단축을 위해 ①밀리미터파 통신이 가능한 주파수 확장, ②스몰셀(Small cell)을 도입한 기지국, ③다중안테나 송수신(Massive MIMO), ④네트워크 슬라이싱(Network Slicing) 등의 기술을 도입함[5G 주요 요소기술 특징]
자료: 삼정 KPMG25,000
20,000
15,000
10,000
RO
5,000
2017
2018
2019
2020
2021
4G/5G
(loT,M2M)
■吾号MassiveMMO+
4G5G
（）
4G
4G
5G
2.6Ghz
品号
3.5Ghz
28Ghz
（m）
5G
（1k0H）
loboly
Massive
（km）
loT
5G3.5Ghz28Ghz0
MassiveMIMOEI
舍Korea
Credit Information
Services


좀 더 많은 정보가 추출됐지만 사용하기엔 아직 부족하다. 

In [11]:
# PDF 파일 4페이지의 테이블 정보 추출 확인

from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 불러올 객체 PyPDFLoader 선언
loader = PyPDFLoader(filename)

# PDF 파일 로드 및 페이지별로 자르기
pages = loader.load_and_split()
print(pages[3])    

page_content='| 4 | CIS이슈리포트 2022-2호 
[혁신성장 ICT 산업 정책금융 공급 현황]                                                            (단위: 억 원, 괄호는 점유율 %)구분2017년 말2018년 말2019년 말2020년 말2021년 말혁신성장 ICT 산업45,075 72,799 81,805 139,687 169,089 (18.7)(20.7)(18.5)(20.3)(19.8)정보통신15,658 27,417 39,033 65,324 77,750 (6.5)(7.8)(8.8)(9.5)(9.1)전기전자26,637 38,521 35,922 62,856 77,485 (11.1)(10.9)(8.1)(9.1)(9.1)센서측정2,780 6,861 6,851 11,506 13,854 (1.2)(1.9)(1.5)(1.7)(1.6)혁신성장 정책금융 총 공급액240,787 351,987 443,180 688,409 854,338 3. 정보통신 테마 혁신성장 정책금융 현황 및 관련 산업 동향▶(지원 현황) 정보통신 테마를 구성하는 기술분야별 정책금융 지원 현황 분석결과, 공급점유율 관점에서는 차세대무선통신미디어 분야에 가장 많은 정책자금이 투입 되고 있으며, 공급량 증가율 관점에서는 능동형컴퓨팅 분야로의 정책자금 지원 증가 속도가 가장 빠른 추세임￮차세대무선통신미디어란 전송속도 향상, 소모전력 절감, 고속이동 중 끊김없는 통신 등 새로운 무선환경에 필요한 통신, 인프라 및 서비스 기술을 통칭하며, 4G/5G/6G, 사물인터넷, 방송통신인프라 등의 품목으로 구성됨-정보통신 테마 내 혁신성장 정책금융 공급 규모의 약 50%를 점유하고 있으며, 이는 초연결 미래사회를 구축하기 위해 네트워크 기반 기술 사업화에 대한 정책자금 공급이 꾸준함에 따른 것으로 분석됨￮능동형컴퓨팅이란 거대하고 복잡해지는 데이터의 효율적 가공과 관리를 위한 인간두뇌와 유사한 형태의 정보처리기술을 말하며, 인공지능, 상황인지컴퓨팅 등의 품

PyPDFium2  
[이미지+텍스트 페이지 내 텍스트 추출]

In [14]:
from langchain.document_loaders import PyPDFium2Loader

import warnings 
warnings.filterwarnings("ignore")

loader = PyPDFium2Loader(filename) 

data = loader.load()
print(data[5].page_content)

| 6 | CIS이슈리포트 2022-2호 
▶ (주요품목① : 5G 이동통신) 정보통신 테마 내 기술분야 중 혁신성장 정책금융 공급규모가 가장 큰
차세대무선통신미디어 분야의 경우 4G/5G 기술품목의 정책금융 공급 비중이 가장 높은 것으로 확인됨
[차세대무선통신미디어 분야 내 기술품목별 혁신성장 정책금융 공급액 추이]
(단위: 억 원)
▶ 5G 이동통신 시스템은 ITU(International Telecommunication Union)가 정의한 5세대 이동통신
규격을 만족시키는 무선 이동통신 네트워크 기술로, 2019년부터 국내 서비스를 시작함
￮ 4G 이동통신 시스템(LTE)과 비교할 때 전송속도의 향상(1Gbps→20Gbps), 이동성 향상
(350km/h→500km/h에서 끊김없는 데이터 전송 가능), 최대 연결가능 기기수 증가(10만 대 →
100만 대 이상), 데이터 전송지연 감소(10ms→1ms) 등의 향상된 기능을 제공함
￮ 5G는 전송속도 향상, 다수기기 접속 및 지연시간 단축을 위해 ①밀리미터파 통신이 가능한 주파수 
확장, ②스몰셀(Small cell)을 도입한 기지국, ③다중안테나 송수신(Massive MIMO), ④네트워크 
슬라이싱(Network Slicing) 등의 기술을 도입함
[5G 주요 요소기술 특징]
자료: 삼정 KPMG



In [15]:
# 테이블 추출 

from langchain.document_loaders import PyPDFium2Loader 

loader = PyPDFium2Loader(filename)

data = loader.load()
print(data[3].page_content)

| 4 | CIS이슈리포트 2022-2호 
[혁신성장 ICT 산업 정책금융 공급 현황]
 (단위: 억 원, 괄호는 점유율 %)
구분 2017년 말 2018년 말 2019년 말 2020년 말 2021년 말
혁신성장 ICT 산업
45,075 72,799 81,805 139,687 169,089 
(18.7) (20.7) (18.5) (20.3) (19.8)
정보통신
15,658 27,417 39,033 65,324 77,750 
(6.5) (7.8) (8.8) (9.5) (9.1)
전기전자
26,637 38,521 35,922 62,856 77,485 
(11.1) (10.9) (8.1) (9.1) (9.1)
센서측정
2,780 6,861 6,851 11,506 13,854 
(1.2) (1.9) (1.5) (1.7) (1.6)
혁신성장 정책금융 
총 공급액 240,787 351,987 443,180 688,409 854,338 
3. 정보통신 테마 혁신성장 정책금융 현황 및 관련 산업 동향
▶ (지원 현황) 정보통신 테마를 구성하는 기술분야별 정책금융 지원 현황 분석결과, 공급점유율 관점
에서는 차세대무선통신미디어 분야에 가장 많은 정책자금이 투입 되고 있으며, 공급량 증가율 관점
에서는 능동형컴퓨팅 분야로의 정책자금 지원 증가 속도가 가장 빠른 추세임
￮ 차세대무선통신미디어란 전송속도 향상, 소모전력 절감, 고속이동 중 끊김없는 통신 등 새로운 무선
환경에 필요한 통신, 인프라 및 서비스 기술을 통칭하며, 4G/5G/6G, 사물인터넷, 방송통신인프라 
등의 품목으로 구성됨
- 정보통신 테마 내 혁신성장 정책금융 공급 규모의 약 50%를 점유하고 있으며, 이는 초연결 미래
사회를 구축하기 위해 네트워크 기반 기술 사업화에 대한 정책자금 공급이 꾸준함에 따른 것으로 
분석됨
￮ 능동형컴퓨팅이란 거대하고 복잡해지는 데이터의 효율적 가공과 관리를 위한 인간두뇌와 유사한 
형태의 정보처리기술을 말하며, 인공지능, 상황인지컴퓨팅 등의 품목으로 구성됨
- 컴퓨팅 기술을 활

PyPDFLoader vs PyPDFium2Loader  

[PyPDFLoader의 텍스트 추출 소요 시간]

In [16]:
from langchain.document_loaders import PyPDFLoader
import time 

start_time = time.time()
loader = PyPDFLoader(filename)
pages = loader.load_and_split()
end_time = time.time()
print("실행 시간: ", end_time - start_time)

실행 시간:  1.9890124797821045


In [17]:
from langchain.document_loaders import PyPDFium2Loader
import time

start_time = time.time()
loader = PyPDFium2Loader(filename)
pages = loader.load()
end_time = time.time()
print("실행 시간: ", end_time - start_time)

실행 시간:  0.0859677791595459
